<a href="https://colab.research.google.com/github/Djaxis/MY-Python-Evolution/blob/main/D%C3%A9but_du_script_Python_Scrap_Youtube_Sentimet_Analysis_Brice_De_Campos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chrome > permet de créer une instance d'un navigateur Chrome.

By     > est utilisé pour sélectionner des éléments de la page web en utilisant différents critères.

Keys   > permet de simuler des touches du clavier, telles que "Entrée" ou "Echap".

WebDriverWait > est utilisé pour attendre qu'un élément spécifié soit présent sur la page avant de continuer le script.

expected_conditions >  est utilisé pour définir les conditions attendues pour qu'une action soit effectuée.

In [ ]:
!pip install selenium

Le script Python ne trouve pas le fichier du driver dans le répertoire courant. Dans ce cas, vous pouvez soit :

Copier le fichier du driver dans le répertoire courant où se trouve votre script Python.

Indiquer le chemin absolu du fichier du driver. Vous pouvez obtenir le chemin absolu en utilisant la commande pwd dans le terminal (ou cmd sous Windows) pour obtenir le chemin courant, puis ajouter le nom du fichier du driver. Par exemple, si le chemin courant est /home/user/Documents, et que le fichier du driver est dans /home/user/Downloads/chromedriver, alors vous pouvez utiliser executable_path='/home/user/Downloads/chromedriver'.

Indiquer le chemin relatif du fichier du driver par rapport au répertoire courant où se trouve votre script Python. Par exemple, si le fichier du driver est dans le répertoire parent (..) de votre script Python, vous pouvez utiliser executable_path='../chromedriver'.

Notez également que l'utilisation de executable_path est déconseillée, il est préférable d'utiliser Service comme je vous l'ai montré dans la solution précédente.

In [ ]:
!ls /content/drive/MyDrive/BDD/
from google.colab import drive
import os.path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import json
import numpy as np
import plotly.express as px
import requests
import seaborn as sns
import urllib.request
import matplotlib.pyplot as plt


import os
# Chemin vers le driver
driver_path = "./chromedriver"
# Vérifier si le fichier existe
if os.path.exists(driver_path):
    print("Le driver est dans le même répertoire que le script Python")
else:
    print("Le driver n'est pas dans le même répertoire que le script Python")


# monter votre Google Drive
from google.colab import drive
drive.mount('/content/drive')

# vérifier si le dossier 'BDD' existe dans votre Google Drive
bdd_path = '/content/drive/MyDrive/BDD'
if not os.path.exists(bdd_path):
    os.mkdir(bdd_path)

# vérifier si le fichier 'scrapyoutube2023.csv' existe dans le dossier 'BDD' de votre Google Drive
csv_path = '/content/drive/MyDrive/BDD/scrapyoutube2023.csv'
if not os.path.exists(csv_path):
    # si le fichier n'existe pas, créer un nouveau dataframe avec le nom "scrapyoutube2023"
    scrapyoutube2023 = pd.DataFrame()
    # enregistrer le dataframe dans le fichier CSV
    scrapyoutube2023.to_csv(csv_path, index=False)
    print(f'Le fichier {csv_path} a été créé.')

# initialisation du driver Selenium
options = Options()
options.headless = True 
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(executable_path='./chromedriver', options=options)

url="https://www.youtube.com/watch?v=rnDbi8gti0U&ab_channel=Underscore_" # url adresse 
html = requests.get(url) # requete de la page 
html # REPONSE 406


# 3 - Contourne la protection en indiquant un navigateur existant. Et relance ta requête.
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
html = requests.get(url, headers={'User-Agent': navigator})
html # REPONSE 200 C'est ok

url = "https://www.youtube.com/watch?v=rnDbi8gti0U&ab_channel=Underscore_"
driver.get(url)

# attendre que la section de commentaires soit chargée
comment_section = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="comments"]'))
)

# boucle pour scroller et charger tous les commentaires
while True:
    try:
        # cliquer sur le bouton "Afficher plus de commentaires"
        more_btn = driver.find_element_by_xpath('//*[@id="more"]/yt-formatted-string')
        driver.execute_script("arguments[0].scrollIntoView(true);", more_btn)
        more_btn.click()
        # attendre le chargement des commentaires supplémentaires
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.XPATH, '//*[@id="progress"]'))
        )
    except:
        break

# extraire les commentaires de la section de commentaires
comment_elems = driver.find_elements_by_xpath('//*[@id="comment"]')

# stocker les commentaires dans une liste
comment_list = []
for comment_elem in comment_elems:
    comment_list.append(comment_elem.text)

# créer un dataframe pandas à partir de la liste de commentaires
df_comments = pd.DataFrame({'Comment': comment_list})

# ajouter le dataframe de commentaires au fichier CSV
df_comments.to_csv(csv_path, mode='a', index=False, header=not os.path.exists(csv_path))

# imprimer les commentaires
for comment in comment_list:
    print(comment)

# fermer


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://www.youtube.com/watch?v=rnDbi8gti0U&ab_channel=Underscore_"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

comment_list = []

# trouver la section qui contient les commentaires

comment_section = soup.find('yt-formatted-string', {'class': 'count-text style-scope ytd-comments-header-renderer'}).find_parent('div', {'id': 'comments'})
if comment_section:
    # extraire le jeton continu pour la pagination
    continuation_token = comment_section.find('yt-next-continuation')['data-continuation']

    # boucle pour extraire les commentaires de chaque page de pagination
    while continuation_token:
        # construire l'URL de la page de pagination
        url = 'https://www.youtube.com/comment_service_ajax?action_get_comments=1&pbj=1&ctoken=' + continuation_token + '&continuation=' + continuation_token + '&itct=' + itct

        # envoyer une requête GET à l'URL de la page de pagination
        response = requests.get(url, headers=headers)

        # extraire les commentaires de la page de pagination
        comments = extract_comments(response.json())

        # ajouter les commentaires extraits à la liste complète de commentaires
        comment_list.extend(comments)

        # extraire le jeton continu pour la prochaine page de pagination (s'il y en a une)
        continuation_token = extract_continuation_token(response.json())
else:
    print("La section de commentaires n'a pas été trouvée.")



AttributeError: ignored

In [ ]:
url="https://www.youtube.com/watch?v=rnDbi8gti0U&ab_channel=Underscore_" # url adresse 
html = requests.get(url) # requete de la page 
html # REPONSE 406

<Response [200]>

In [ ]:
import requests
from bs4 import BeautifulSoup

# 3 - Contourne la protection en indiquant un navigateur existant. Et relance ta requête.
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
html = requests.get(url, headers={'User-Agent': navigator})
html # REPONSE 200 C'est ok

soup = BeautifulSoup(html.text, 'html.parser') # lecture de la page  avec tous les codes - on crée une soupe avec le code HTML
soup # affiche le code et ses balises de la page.

# Grace à l'inspecteur d'éléments, on trouve la balise permettant d'identifier le cadre des blagues
# chaque commentaire YouTube soit contenu dans une balise ytd-comment-thread-renderer qui contient plusieurs sous-éléments, 
# y compris la balise div avec l'ID body qui contient le texte du commentaire lui-même. 

comments = soup.find_all('div', {'id': 'content-text'})
comment_list = []

for comment in comments:
    comment_list.append(comment.text.strip())

for comment in comment_list:
    print(comment)


In [ ]:
url="https://www.youtube.com/watch?v=rnDbi8gti0U&ab_channel=Underscore_" # url adresse 
html = requests.get(url) # requete de la page 
html # REPONSE 406


# 3 - Contourne la protection en indiquant un navigateur existant. Et relance ta requête.
navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
html = requests.get(url, headers={'User-Agent': navigator})
html # REPONSE 200 C'est ok

from bs4 import BeautifulSoup
soup = BeautifulSoup(html.text, 'html.parser') # lecture de la page  avec tous les codes - on crée une soupe avec le code HTML
soup # affiche le code et ses balises de la page.


# Grace à l'inspecteur d'éléments, on trouve la balise permettant d'identifier le cadre des blagues
# chaque commentaire YouTube soit contenu dans une balise ytd-comment-thread-renderer qui contient plusieurs sous-éléments, 
# y compris la balise div avec l'ID body qui contient le texte du commentaire lui-même. 
comments = soup.find_all('ytd-comment-thread-renderer')


#extraire le texte de chaque commentaire, vous pouvez itérer sur la liste comments 
#et extraire le texte de la balise div avec l'ID body à l'intérieur de chaque élément :
# Voici un exemple de code qui stocke les commentaires dans une liste nommée comment_list et les imprime dans la console Python:
comment_list = []

for comment in comments:
    comment_text = comment.find('div', {'id': 'content-text'}).text.strip()
    comment_list.append(comment_text)

for comment in comment_list:
    print(comment)


In [ ]:
datascrapyoutube= [] # stocker les commentaires extraits.

youtube_video_url= "https://www.youtube.com/watch?v=rnDbi8gti0U&ab_channel=Underscore_"

with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:
      wait = WebDriverWait(driver,15)
      driver.get(youtube_video_url)

 
# une boucle for pour faire défiler la page jusqu'à la fin en utilisant la fonction send_keys pour envoyer la touche "Fin" à la page web. 
# Le script attend ensuite 15 secondes pour que la page se charge, puis répète ce processus 200 fois.
for item in range(200):

      wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
      time.sleep(15)

# Une fois que la page a été complètement chargée, le script utilise "presence_of_all_elements_located" pour attendre que tous les éléments de commentaire soient présents sur la page.
# Il utilise ensuite la méthode text pour extraire le texte de chaque élément de commentaire et l'ajouter à la liste data.
for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):

    datascrapyoutube.append(comment.text)



<ipython-input-34-6f0bec398da2>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:


WebDriverException: ignored

In [ ]:
import pandas as pd 

df = pd.DataFrame(dadatascrapyoutube, columns=['comment'])

df.head()